#TAG SMART EMAILS

GENSIM AND LDA

https://radimrehurek.com/gensim/apiref.html

In [1]:
#Library
from gensim.corpora import Dictionary
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel
import spacy
import pandas as pd

In [2]:
%pwd

'C:\\Users\\HomePC\\tagger-ds'

In [3]:
import requests
url = 'https://raw.githubusercontent.com/Moly-malibu/Tag-Smart-Emails/master/Enronemails.csv?token=AN2RW2GO4AWTXK4OPUDXJV26ZQ3QC'
myfile = requests.get(url)
open('email.csv', 'wb').write(myfile.content)

24004915

In [4]:
#EMAILS FROM ENRON COMPANY SAMPLING
df = pd.read_csv("email.csv")
print(df.shape) 
df.head(7).drop(['Unnamed: 0'], axis=1)

(10000, 3)


,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...
5,allen-p/_sent_mail/1002.,Message-ID: <30965995.1075863688265.JavaMail.e...
6,allen-p/_sent_mail/1003.,Message-ID: <16254169.1075863688286.JavaMail.e...


In [5]:
#TO GET EMAIL BODY
[s.strip() for s in df['message'][1].splitlines()]

['Message-ID: <15464986.1075855378456.JavaMail.evans@thyme>',
 'Date: Fri, 4 May 2001 13:51:00 -0700 (PDT)',
 'From: phillip.allen@enron.com',
 'To: john.lavorato@enron.com',
 'Subject: Re:',
 'Mime-Version: 1.0',
 'Content-Type: text/plain; charset=us-ascii',
 'Content-Transfer-Encoding: 7bit',
 'X-From: Phillip K Allen',
 'X-To: John J Lavorato <John J Lavorato/ENRON@enronXgate@ENRON>',
 'X-cc:',
 'X-bcc:',
 "X-Folder: \\Phillip_Allen_Jan2002_1\\Allen, Phillip K.\\'Sent Mail",
 'X-Origin: Allen-P',
 'X-FileName: pallen (Non-Privileged).pst',
 '',
 'Traveling to have a business meeting takes the fun out of the trip.  Especially if you have to prepare a presentation.  I would suggest holding the business plan meetings here then take a trip without any formal business meetings.  I would even try and get some honest opinions on whether a trip is even desired or necessary.',
 '',
 'As far as the business meetings, I think it would be more productive to try and stimulate discussions across

In [6]:
email = ' '.join([s.strip() for s in df['message'][1].splitlines()][15:])

In [7]:
#CLEAN EMAIL BODY
def cleanemail(email):
    return ' '.join([s.strip() for s in email.splitlines()][15:])

df['email_body'] = df['message'].apply(cleanemail)

df.head()

,Unnamed: 0,file,message,email_body
0,0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...,Here is our forecast
1,1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...,Traveling to have a business meeting takes th...
2,2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...,test successful. way to go!!!
3,3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...,"Randy, Can you send me a schedule of the sal..."
4,4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...,Let's shoot for Tuesday at 11:45.


In [8]:
#CLEAN HTML
import re
import html

def cleanup_html(raw_html):
    clean_html = re.sub(r'<.*?>', '', raw_html)
    clean_html_http = re.sub(r'http\S+([\.]{3})?', '', clean_html)
    clean_txt = html.unescape(clean_html)
    return clean_txt
df['email_body'] = df['email_body'].apply(cleanup_html)
df.sample(10)
for row in df['email_body'].sample(10):
    print(row)
    print()

 Adrianne,  I spoke to Brent D.  I would recommend that he is given a chance to interview for the program.  Phillip  -----Original Message----- From: Engler, Adrianne Sent: Monday, October 29, 2001 8:10 AM To: Allen, Phillip K. Subject: RE: the candidate we spoke about this morning...  Phillip -  Lets try this one....Thanks!  Adrianne -----Original Message----- From: Allen, Phillip K. Sent: Monday, October 29, 2001 10:08 AM To: Engler, Adrianne Subject: RE: the candidate we spoke about this morning... Adrianne,  I cannot download his resume.  Please resend.  Phillip  -----Original Message----- From: Engler, Adrianne Sent: Monday, October 29, 2001 7:34 AM To: Allen, Phillip K. Subject: the candidate we spoke about this morning...     Phillip -  This is the candidate I spoke with you about this morning...  Brent currently works in our London office as an Analyst and is looking to come over to the Houston office.  We would like to get him into the November 1st interviews.  His contact inf

In [9]:
#CLEAN DATA
#remove whitespace
df['email_body'] = df['email_body'].str.strip().str.lower()

#Start with date
df['email_body'].str.match('\d?\d/\d?\d/\d{4}').all()

#Replace all non-overlapping matches
df['email_body'] = df['email_body'].str.replace('[^a-zA-Z\s]', '').str.replace('\s+', ' ')

In [10]:
import nltk
import string
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HomePC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
#Getting Stopwords
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer

# define additional stop words
import nltk
import spacy

nlp = spacy.load('en_core_web_lg')
my_stop_words = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 
                 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 
                 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 
                 "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 
                 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 
                 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 
                 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 
                 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 
                 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 
                 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 
                 'into', 'through', 'during', 'before', 'after', 'above', 'below',
                 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 
                 'again', 'further', 'then', 'once', 'here', 'there', 'when', 
                 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 
                 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 
                 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 
                 'can', 'will', 'just', 'don', "don't", 'should', "should've", 
                 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', 
                 "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', 
                 "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', 
                 "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 
                 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 
                 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', 
                 "weren't", 'won', "won't", 'wouldn', "wouldn't"]

STOP_WORDS = nlp.Defaults.stop_words.union(my_stop_words)

stop = stopwords.words('english')
sno = SnowballStemmer('english')
print (stop)
print('***************************************')
print(sno.stem('tasty'))

STOP_WORDS = list(STOP_WORDS)
print('\n'.join([word for word in STOP_WORDS[:10]]))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [13]:
tokens = []
for doc in nlp.pipe(df['email_body'], batch_size=500):

    doc_tokens = []

    for token in doc:
        if (token.is_stop == False) & (token.is_punct == False):
            doc_tokens.append(token.lemma_.lower())

    tokens.append(doc_tokens)

df['tokens'] = tokens

In [14]:
df['tokens'].head()

0                                           [forecast]
1    [travel, business, meeting, take, fun, trip, e...
2                              [test, successful, way]
3    [randy, send, schedule, salary, level, schedul...
4                                [let, shoot, tuesday]
Name: tokens, dtype: object

In [15]:
id2word = Dictionary(df['tokens'])
print(len(id2word))

45973


In [16]:
id2word.filter_extremes(no_below=5, no_above=.98)
print(len(id2word))

14978


In [17]:
#https://radimrehurek.com/gensim/corpora/dictionary.html

corpus = [id2word.doc2bow(d) for d in df['tokens']]

In [18]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   random_state=723812,
                   num_topics = 15,
                   passes=10,
                   workers=12
                  )

In [19]:
model = LdaMulticore(corpus=corpus, num_topics=10, id2word=id2word, workers=12, passes=5)

In [20]:
import re

words = [re.findall(r'"([^"]*)"',t[1]) for t in model.print_topics()]

In [21]:
topics = [' '.join(t[0:5]) for t in words]

In [22]:
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n")

------ Topic 0 ------
company subject know phillip not
------ Topic 1 ------
email john subject pm active
------ Topic 2 ------
pm enron subject john market
------ Topic 3 ------
enron subject send message original
------ Topic 4 ------
gas meeting energy enron say
------ Topic 5 ------
enron say company market year
------ Topic 6 ------
image phillip pm enron thank
------ Topic 7 ------
market pm iso price schedule
------ Topic 8 ------
pm phillip subject john cc
------ Topic 9 ------
john message subject cc pm


In [23]:
!pip install pyLDAvis
import pyLDAvis

In [24]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

pyLDAvis.gensim.prepare(model, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.120947 -0.030960       1        1  24.999735
0     -0.031014 -0.031000       2        1   9.903051
2      0.002150  0.020992       3        1   9.880755
4     -0.085184  0.041098       4        1   9.029905
7     -0.047685  0.033387       5        1   8.944244
1      0.146193  0.062845       6        1   8.852551
6      0.033747  0.016961       7        1   8.605923
3     -0.021345 -0.031686       8        1   8.375796
8      0.028597  0.034368       9        1   5.905430
9      0.095488 -0.116005      10        1   5.502608, topic_info=         Term         Freq         Total Category  logprob  loglift
77       john  7153.000000   7153.000000  Default  30.0000  30.0000
2674    image  3497.000000   3497.000000  Default  29.0000  29.0000
1409  message  4292.000000   4292.000000  Default  28.0000  28.0000
959     email  5925.000000   5925.000000  Default  27.0000  27.0000
193   subject  7597.000000   7597.000000  Default  26.0000  26.0000
...       ...          ...           ...      ...      ...      ...
975      week   186.606384   3081.867676  Topic10  -5.8598   0.0957
37      think   182.262833   2538.897217  Topic10  -5.8833   0.2659
139   forward   180.942093   3166.180664  Topic10  -5.8906   0.0378
208     enron   189.889435  11272.192383  Topic10  -5.8423  -1.1837
174     power   181.828918   4235.522949  Topic10  -5.8857  -0.2482

[922 rows x 6 columns], token_table=      Topic      Freq Term
term                      
90        1  0.045275     
90        2  0.091404     
90        3  0.093967     
90        4  0.044421     
90        5  0.075174     
...     ...       ...  ...
6311      5  0.160761  zip
6311      6  0.617659  zip
6311      7  0.016922  zip
6311      8  0.008461  zip
6311      9  0.008461  zip

[4715 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 1, 3, 5, 8, 2, 7, 4, 9, 10])

In [25]:
#'corpus'

distro = [lda[d] for d in corpus]

num_topics = 20

def update(doc):
        d_dist = {k:0 for k in range(0,num_topics)}
        for topic in doc:
            d_dist[topic[0]] = topic[1]
        return d_dist
    
new_distro = [update(d) for d in distro]

In [26]:
#Correlated
topics = pd.DataFrame.from_records(new_distro)
topics.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.033339,0.033338,0.533262,0.033338,0.033338,0.033339,0.033338,0.033338,0.033338,0.033338,0.033338,0.033338,0.033338,0.033338,0.033338,0,0,0,0,0
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.254033,0.057943,0.000000,0.000000,0.000000,0.000000,0.000000,0.674670,0.000000,0,0,0,0,0
2,0.016672,0.016672,0.016672,0.016672,0.016672,0.016672,0.016672,0.016672,0.016672,0.016672,0.016672,0.016672,0.545097,0.238171,0.016672,0,0,0,0,0
3,0.000000,0.000000,0.000000,0.000000,0.503629,0.000000,0.442185,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0
4,0.016670,0.016670,0.016670,0.016670,0.016670,0.016670,0.766621,0.016670,0.016670,0.016670,0.016670,0.016670,0.016670,0.016670,0.016670,0,0,0,0,0


In [27]:
df['email_body'] = topics.idxmax(axis=1)

In [28]:
#Topic  
df['email_body'].describe()

count    10000.000000
mean         8.099300
std          3.886174
min          0.000000
25%          5.000000
50%          8.000000
75%         12.000000
max         14.000000
Name: email_body, dtype: float64

In [29]:
df['email_body'].value_counts()

13    1737
9     1434
6     1064
5      893
4      857
8      782
12     717
1      462
10     440
14     392
7      290
11     257
3      242
2      218
0      215
Name: email_body, dtype: int64

##Create the Emails Tags

##tfidf term frequency–inverse document frequency


In [30]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HomePC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HomePC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [31]:
from nltk.tokenize import word_tokenize

# split into words
tokens = word_tokenize(email)

# convert to lower case
tokens = [w.lower() for w in tokens]

# prepare regex for char filtering
re_punc = re.compile('[%s]' % re.escape(string.punctuation))

# remove punctuation from each word
stripped = [re_punc.sub('', w) for w in tokens]

# remove remaining tokens that are not alphabetic
words = [word for word in stripped if word.isalpha()]

# filter out stop words
stop_words = set(stopwords.words('english'))
words = [w for w in words if not w in stop_words]
print(words[:200])

['traveling', 'business', 'meeting', 'takes', 'fun', 'trip', 'especially', 'prepare', 'presentation', 'would', 'suggest', 'holding', 'business', 'plan', 'meetings', 'take', 'trip', 'without', 'formal', 'business', 'meetings', 'would', 'even', 'try', 'get', 'honest', 'opinions', 'whether', 'trip', 'even', 'desired', 'necessary', 'far', 'business', 'meetings', 'think', 'would', 'productive', 'try', 'stimulate', 'discussions', 'across', 'different', 'groups', 'working', 'often', 'presenter', 'speaks', 'others', 'quiet', 'waiting', 'turn', 'meetings', 'might', 'better', 'held', 'round', 'table', 'discussion', 'format', 'suggestion', 'go', 'austin', 'play', 'golf', 'rent', 'ski', 'boat', 'jet', 'ski', 'flying', 'somewhere', 'takes', 'much', 'time']


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count_vec = CountVectorizer(stop_words="english", analyzer='word', 
                            ngram_range=(1, 2), max_df=1.0, min_df=1, max_features=None)

count_train = count_vec.fit(words)
bag_of_words = count_vec.transform(words)

print(count_vec.get_feature_names())

['austin', 'better', 'boat', 'business', 'desired', 'different', 'discussion', 'discussions', 'especially', 'far', 'flying', 'formal', 'format', 'fun', 'golf', 'groups', 'held', 'holding', 'honest', 'jet', 'meeting', 'meetings', 'necessary', 'opinions', 'plan', 'play', 'prepare', 'presentation', 'presenter', 'productive', 'quiet', 'rent', 'round', 'ski', 'speaks', 'stimulate', 'suggest', 'suggestion', 'table', 'takes', 'think', 'time', 'traveling', 'trip', 'try', 'turn', 'waiting', 'working']


In [ ]:
text_freque = TfidfVectorizer(smooth_idf=False, sublinear_tf=False, norm=None, analyzer='word')
txt_fitted = text_freque.fit(words)
txt_transformed = txt_fitted.transform(words)
print ("The text: ", words)

The text:  ['traveling', 'business', 'meeting', 'takes', 'fun', 'trip', 'especially', 'prepare', 'presentation', 'would', 'suggest', 'holding', 'business', 'plan', 'meetings', 'take', 'trip', 'without', 'formal', 'business', 'meetings', 'would', 'even', 'try', 'get', 'honest', 'opinions', 'whether', 'trip', 'even', 'desired', 'necessary', 'far', 'business', 'meetings', 'think', 'would', 'productive', 'try', 'stimulate', 'discussions', 'across', 'different', 'groups', 'working', 'often', 'presenter', 'speaks', 'others', 'quiet', 'waiting', 'turn', 'meetings', 'might', 'better', 'held', 'round', 'table', 'discussion', 'format', 'suggestion', 'go', 'austin', 'play', 'golf', 'rent', 'ski', 'boat', 'jet', 'ski', 'flying', 'somewhere', 'takes', 'much', 'time']


In [ ]:
idfreque = text_freque.idf_
print(dict(zip(txt_fitted.get_feature_names(), idfreque)))

{'across': 5.31748811353631, 'austin': 5.31748811353631, 'better': 5.31748811353631, 'boat': 5.31748811353631, 'business': 3.9311937524164198, 'desired': 5.31748811353631, 'different': 5.31748811353631, 'discussion': 5.31748811353631, 'discussions': 5.31748811353631, 'especially': 5.31748811353631, 'even': 4.624340932976365, 'far': 5.31748811353631, 'flying': 5.31748811353631, 'formal': 5.31748811353631, 'format': 5.31748811353631, 'fun': 5.31748811353631, 'get': 5.31748811353631, 'go': 5.31748811353631, 'golf': 5.31748811353631, 'groups': 5.31748811353631, 'held': 5.31748811353631, 'holding': 5.31748811353631, 'honest': 5.31748811353631, 'jet': 5.31748811353631, 'meeting': 5.31748811353631, 'meetings': 3.9311937524164198, 'might': 5.31748811353631, 'much': 5.31748811353631, 'necessary': 5.31748811353631, 'often': 5.31748811353631, 'opinions': 5.31748811353631, 'others': 5.31748811353631, 'plan': 5.31748811353631, 'play': 5.31748811353631, 'prepare': 5.31748811353631, 'presentation': 5

In [ ]:
text_freque.vocabulary_

{'across': 0,
 'austin': 1,
 'better': 2,
 'boat': 3,
 'business': 4,
 'desired': 5,
 'different': 6,
 'discussion': 7,
 'discussions': 8,
 'especially': 9,
 'even': 10,
 'far': 11,
 'flying': 12,
 'formal': 13,
 'format': 14,
 'fun': 15,
 'get': 16,
 'go': 17,
 'golf': 18,
 'groups': 19,
 'held': 20,
 'holding': 21,
 'honest': 22,
 'jet': 23,
 'meeting': 24,
 'meetings': 25,
 'might': 26,
 'much': 27,
 'necessary': 28,
 'often': 29,
 'opinions': 30,
 'others': 31,
 'plan': 32,
 'play': 33,
 'prepare': 34,
 'presentation': 35,
 'presenter': 36,
 'productive': 37,
 'quiet': 38,
 'rent': 39,
 'round': 40,
 'ski': 41,
 'somewhere': 42,
 'speaks': 43,
 'stimulate': 44,
 'suggest': 45,
 'suggestion': 46,
 'table': 47,
 'take': 48,
 'takes': 49,
 'think': 50,
 'time': 51,
 'traveling': 52,
 'trip': 53,
 'try': 54,
 'turn': 55,
 'waiting': 56,
 'whether': 57,
 'without': 58,
 'working': 59,
 'would': 60}

In [ ]:
words_dist = dict(zip(txt_fitted.get_feature_names(), idfreque))
words_dist

{'across': 5.31748811353631,
 'austin': 5.31748811353631,
 'better': 5.31748811353631,
 'boat': 5.31748811353631,
 'business': 3.9311937524164198,
 'desired': 5.31748811353631,
 'different': 5.31748811353631,
 'discussion': 5.31748811353631,
 'discussions': 5.31748811353631,
 'especially': 5.31748811353631,
 'even': 4.624340932976365,
 'far': 5.31748811353631,
 'flying': 5.31748811353631,
 'formal': 5.31748811353631,
 'format': 5.31748811353631,
 'fun': 5.31748811353631,
 'get': 5.31748811353631,
 'go': 5.31748811353631,
 'golf': 5.31748811353631,
 'groups': 5.31748811353631,
 'held': 5.31748811353631,
 'holding': 5.31748811353631,
 'honest': 5.31748811353631,
 'jet': 5.31748811353631,
 'meeting': 5.31748811353631,
 'meetings': 3.9311937524164198,
 'might': 5.31748811353631,
 'much': 5.31748811353631,
 'necessary': 5.31748811353631,
 'often': 5.31748811353631,
 'opinions': 5.31748811353631,
 'others': 5.31748811353631,
 'plan': 5.31748811353631,
 'play': 5.31748811353631,
 'prepare': 5

In [ ]:
import numpy as np

# get feature names
feature_names = np.array(text_freque.get_feature_names())
sorted_by_idf = np.argsort(text_freque.idf_)
print("Features with lowest id frequence:\n{}".format(feature_names[sorted_by_idf[:3]]))
print("\nFeatures with highest id frequence:\n{}".format(feature_names[sorted_by_idf[-3:]]))

Features with lowest id frequence:
['business' 'meetings' 'would']

Features with highest id frequence:
['often' 'get' 'opinions']


##TFID VECTORIZER  - Transforms text to feature vectors


In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Instantiate vectorizer object
tfidf = TfidfVectorizer(stop_words='english', max_features=5000)

# Create a vocabulary and get word counts per document
data = tfidf.fit_transform(words) # Similiar to fit_predict

# Print word counts

# Get feature names to use as dataframe column headers
data = pd.DataFrame(data.todense(), columns=tfidf.get_feature_names())

# View Feature Matrix as DataFrame
data.head()

,austin,better,boat,business,desired,different,discussion,discussions,especially,far,...,table,takes,think,time,traveling,trip,try,turn,waiting,working
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
#TOKENIZER AND LEMMA
def tokenize(document):
    
    doc = nlp(document)
    
    return [token.lemma_.strip() for token in doc if (token.is_stop != True) and (token.is_punct != True)]

In [34]:
# Instantiate vectorizer object
tfidf = TfidfVectorizer(stop_words='english', 
                        ngram_range=(1,2),
                        max_df=.97,
                        min_df=3,
                        tokenizer=tokenize)

# Create a vocabulary and get word counts per document
data = tfidf.fit_transform(words) # Similiar to fit_predict

# Print word counts

# Get feature names to use as dataframe column headers
data = pd.DataFrame(data.todense(), columns=tfidf.get_feature_names())

# View Feature Matrix as DataFrame
data.head()

,business,meeting,trip
0,0.0,0.0,0.0
1,1.0,0.0,0.0
2,0.0,1.0,0.0
3,0.0,0.0,0.0
4,0.0,0.0,0.0


In [35]:
# Calculate Distance of TF-IDF Vectors
from sklearn.metrics.pairwise import cosine_similarity

dist_matrix  = cosine_similarity(dtm)

NameError: name 'dtm' is not defined

In [ ]:
# Turn it into a DataFrame

df = pd.DataFrame(dist_matrix)

In [ ]:
words[5][:300]

'trip'

In [ ]:
print(words[12][:130])

business


In [ ]:
sum(data['business'])

4.0

##Json and CSV

In [ ]:
def save_data_frame_as_csv(df=None, file=None):
    """
    Save a Pandas DataFrame to CSV format.
    """
    try:
        if not file.endswith('.csv'):
            file += '.csv'
        df.to_csv(file, index=False)
        print(f'DataFrame saved as: {file}')
    except Exception as e:
        print("DataFrame couldn't be saved: ", sys.exc_info()[0])
        raise

In [ ]:
save_data_frame_as_csv(df, './merged_data.csv')

In [ ]:
def save_data_frame_as_json(df=None, file=None, orient='records'):
  """
  Save a Pandas DataFrame to Json format.
  """
  try:
    if not file.endswith('.json'):
      file += '.json'
    df.to_json(file, orient=orient)
    print(f'Dataframe saved as: {file}')
  except Exception as e:
    print("Dataframe couldn't be saved: ", sys.exc_info()[0])
    raise

In [ ]:
save_data_frame_as_json(df, '.merged_data.json', orient='records')